## 0. 필요 라이브러리 import

In [ ]:
%pip install google-cloud-storage

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from google.oauth2 import service_account
from google.cloud import storage

## 1. GCP 권한 인증

In [ ]:
## 권한 인증

JSON_KEY_PATH = ".json"

credentials = service_account.Credentials.from_service_account_file(JSON_KEY_PATH)

In [ ]:
## Client 생성

## 2. Parquet 파일 다루기
- 데이터를 객체 저장소에 저장할 때는, 용량 효율성, 빅데이터 도구들과의 호환성, 파티셔닝 등의 이점 때문에 CSV나 JSON 같은 포맷보다 Parquet 포맷을 사용하는 경우가 많습니다. (대부분 Parquet 사용)

In [ ]:
## parquet 파일 쓰기

In [ ]:
## parquet 파일 읽기기

## 3. Bucket 관련 작업

In [ ]:
## 1) 버킷 목록 가져오기

In [ ]:
## 2) 버킷 생성

In [ ]:
## 3) 버킷 정보 조회

In [ ]:
## 4) 버킷 삭제

## 4. 객체 관련 작업

In [ ]:
## 1) 객체 업로드 - 파일

In [ ]:
## 2) 객체 업로드 - DataFrame

In [ ]:
## 3) 객체 다운로드

In [ ]:
## 4) 폴더 내 객체 목록 가져오기

In [ ]:
## 5) 객체 유무 조회

In [ ]:
## 6) 객체 삭제

## 5. Pandas로 GCP 데이터 읽기

In [ ]:
%pip install fsspec gcsfs

Defaulting to user installation because normal site-packages is not writeable
